In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults, reset_defaults
from ipywidgets import embed
from jupyter_cadquery.cad_view import CadqueryView
set_defaults(height = 1000, cad_width = 2500, axes=True, grid=True, axes0=True, ortho=True, transparent=True)
s = cq.StringSyntaxSelector


Overwriting auto display for cadquery Workplane and Shape


In [2]:
#------------------------------------------------------------------------------------------   
# Panasonic HLG103 S-J model

mySTEP_path = "C:\\Users\\Srinivas.G\\OneDrive - Ram Group Global Pte Ltd\\Work_shared\\Technical drawings\\mySTEP\\Spectrometer\\I - HLG1\\"
## Load again to check the output
hlg1 = cq.importers.importStep(f"{mySTEP_path}hl-g103-s-j-trimmed.step")

hlg1

In [3]:
#------------------------------------------------------------------------------------------   
# Mock - up cartridge

cartridge_l = 198/2
cartridge_b =  34.5
cartridge_h = 1
cartridge = cq.Workplane("XY")\
              .box(cartridge_l,cartridge_b,cartridge_h)

cartridge

In [4]:
#------------------------------------------------------------------------------------------   
# Define enclosure class

class baselessShellEnc():
    def __init__(self, outbox_xlen, outbox_ylen , outbox_zlen, wall_t, scrw_d, scrw_edge_clr):
        self.outbox_xlen, self.outbox_ylen ,\
        self.outbox_zlen, self.wall_t,\
        self.scrw_d, self.scrw_edge_clr\
        = (outbox_xlen, outbox_ylen , outbox_zlen, wall_t, scrw_d, scrw_edge_clr)
        self.res, self.inbox_xlen, self.inbox_ylen, self.hole_x, self.hole_y = self.makeEnc()
        
        
    def makeEnc(self):
        inbox_xlen , inbox_ylen =  (self.outbox_xlen - self.wall_t - self.scrw_edge_clr  ,
                                    self.outbox_ylen - self.wall_t - self.scrw_edge_clr )
        hole_x, hole_y  = (self.outbox_xlen - 2*self.scrw_edge_clr  - self.scrw_d,
                           self.outbox_xlen - 2*self.scrw_edge_clr  - self.scrw_d)

        result = cq.Workplane("front").box(self.outbox_xlen, self.outbox_ylen , self.outbox_zlen)\
                                    .faces(">Z").rect(hole_x, hole_y, forConstruction = True)\
                                    .vertices().circle(self.scrw_d).cutThruAll()
        result = result.faces(">Z").workplane().rect(inbox_xlen, inbox_ylen).cutThruAll()
        result = result.faces(">X").workplane().center(0,-outbox_xlen/2)\
                                    .rect(hole_x, hole_y, forConstruction = True)\
                                    .vertices().circle(self.scrw_d).cutThruAll()
        result = result.faces(">Y").workplane().center(outbox_xlen/2,0).rect(hole_x, hole_y,
                                                                             forConstruction = True)\
                                    .vertices().circle(self.scrw_d).cutThruAll()
        
        return (result, inbox_xlen, inbox_ylen, hole_x, hole_y)

In [5]:
#------------------------------------------------------------------------------------------   
# Main enclosure

outbox_xlen, outbox_ylen , outbox_zlen, wall_t, scrw_d, scrw_edge_clr = (250,250,250, 20, 3.05 , 4)
enc = baselessShellEnc(outbox_xlen, outbox_ylen , outbox_zlen, wall_t, scrw_d, scrw_edge_clr)
res = enc.res

#------------------------------------------------------------------------------------------   
# General purpose shelf 1 - Misc.
shelf_h = 15
shelf_l = enc.inbox_xlen 
shelf_b = 70
shelf_loc = -wall_t/2 
res2 = res.faces(">X").workplane().center(-outbox_xlen/2,
                                          shelf_loc).rect(shelf_l,
                                                          shelf_h)\
                                        .extrude(-shelf_b - wall_t)

#------------------------------------------------------------------------------------------   

# General purpose shelf 2 - Raspberry Pi

shelf_h2 = shelf_h
shelf_l2 = shelf_l / 2
shelf_b2 = 80
outbox_xlen = enc.outbox_xlen
shelf3_zoffset =90
shelf3_yoffset = - outbox_xlen/2 - wall_t/2
shelf_h3 = shelf_h
shelf_l3 = shelf_l /2
shelf_b3 = shelf_b/2


res3 = res2.faces("<X").workplane().center(0, 0)\
            .rect(shelf_l2, shelf_h2).extrude(-shelf_b2 - wall_t)

#------------------------------------------------------------------------------------------   
## Shelf for HLG1 breakout board

extr = -shelf_b3 - wall_t
res4 = res3.faces("<X").workplane().center(shelf3_yoffset
                                           + shelf_l3/2 
                                           + wall_t ,
                                           shelf3_zoffset
                                          ).rect(shelf_l3,
                                                 shelf_h3).extrude(extr)

#------------------------------------------------------------------------------------------   
 ## Slot for Cartridge   

laser_z_clr = 25 #  Distance from the center of emission line to surface of bare sensor.    

hlg1_laser_e_d = 12.5 #nearest edge
mhole_e_offy = 3.8 # fixture hole dist from edge
mhole_e_offx = 4.3
mhole_diam = 4.5 # mounting hole
hlg1_height = 60
hlg1_len = 57 
hlg1_base = 20 

cartridge_clr = 0.2
res5 = res4.faces(">Y").workplane().center(-outbox_xlen/2 ,-shelf3_zoffset - shelf_loc)\
                       .rect(cartridge_b + 2*cartridge_clr ,2*cartridge_h, forConstruction = True)\
                       .slot2D(cartridge_b + 2*cartridge_clr,  2*cartridge_h,0).cutBlind(-wall_t)
#------------------------------------------------------------------------------------------   
## HLG1 mounting holes

res6 = res5.faces(">Y").workplane().center(hlg1_height/2 - hlg1_laser_e_d,
                                           laser_z_clr + hlg1_len/2 - cartridge_h)\
                            .rect(hlg1_height - 2*mhole_e_offx, hlg1_len - 2*mhole_e_offy,
                            forConstruction = True)\
                            .vertices().circle(mhole_diam/2).cutBlind(-wall_t)

#------------------------------------------------------------------------------------------   
## General purpose shelf 3 - USB - RS485

res7 =res6.faces("<X").workplane().center(shelf_l3/2 + wall_t/2,
                                          -(laser_z_clr + hlg1_len/2 - cartridge_h) 
                                          + shelf3_zoffset/2 )
res8 = res7.rect(shelf_l3, shelf_h3).extrude(extr)




In [6]:
res8

In [7]:
#------------------------------------------------------------------------------------------   
## Features on upper GP shelves - cable feedthroughs and tie posts

cab_slot_xlen, cab_slot_ylen =  (0.7*shelf_b3, 0.7*shelf_h)
cab_slot_clr = 5 + cab_slot_ylen/2


res9 = res8.faces(">>Z[-4]").workplane()\
           .center( shelf_b3/2 + cab_slot_xlen/2 , shelf_l3/2 - cab_slot_clr)\
           .rect(cab_slot_xlen, cab_slot_ylen).cutThruAll()

res9 = res9.faces(">>Z[-4]").workplane()\
                            .center(0,-shelf_l3 + 2*cab_slot_clr + cab_slot_ylen/2 )\
                            .rect(cab_slot_xlen, cab_slot_ylen).cutThruAll()
res9 = res9.faces(">>Z[-4]").workplane()\
                            .center(0,shelf_l3/2 - cab_slot_ylen/2 )\
                            .rect(cab_slot_xlen, cab_slot_ylen, forConstruction = True)\
                            .vertices().hole(4.1)

# #------------------------------------------------------------------------------------------   
# ## Features on GP shelf 2- cable feedthroughs and tie posts

rpi_zw_xhole,rpi_zw_yhole = (58,23)
rpi_zw_hole_d = 6.1
res9 = res9.faces(">>Z[7]").workplane(origin = (-enc.outbox_xlen/2 + shelf_b2/2 + wall_t ,0))\
                           .rect(rpi_zw_yhole, rpi_zw_xhole, forConstruction = True).vertices()\
                           .hole(rpi_zw_hole_d/2)

rpi_4B_xhole,rpi_4B_yhole = (85,49)
rpi_4B_hole_d = 6.1
res9 = res9.faces(">>Z[7]").workplane(origin = (-enc.outbox_xlen/2 + shelf_b2/2 + wall_t, 0))\
                           .rect(rpi_4B_yhole, rpi_4B_xhole, forConstruction = True)\
                           .vertices().hole(rpi_4B_hole_d/2)

# #------------------------------------------------------------------------------------------   
# ## Features on GP shelf 1,2- cable feedthroughs and lofts

loft_bxlen, loft_bylen = (5,5)
sup_brak_xlen, sup_brak_ylen =  (wall_t/2,shelf_b2)
sup_brak_xlen1, sup_brak_ylen1 =  (wall_t/2,shelf_b-wall_t/2)



res10 = res9.faces(s(">>Z[4]")).workplane().center(-wall_t/4,0)\
                               .box(sup_brak_ylen + wall_t/2, sup_brak_xlen, shelf_h2)\
                               .faces("<<Z[-5]")\
                               .workplane().center(0,0).rect(sup_brak_ylen+ wall_t/2,
                                                             sup_brak_xlen)\
                               .workplane(origin = (-1.5*sup_brak_ylen , 0),
                                          offset = enc.outbox_zlen/2\
                                                   -  shelf3_zoffset - shelf_h3)\
                               .rect(loft_bxlen, loft_bylen ).loft(combine =True)

res11 = res10.faces(s(">>Z[7]")).last()\
             .workplane(origin= ((outbox_xlen/2 - shelf_b/2 - wall_t/2),0))\
             .box(sup_brak_ylen1+ wall_t/2, sup_brak_xlen1, shelf_h2)\
             .faces(">>Z[6]")
res11 = res11.workplane(origin = (enc.outbox_xlen/2 - shelf_b/2 - wall_t/2,0))\
             .rect(sup_brak_ylen1+ wall_t/2, sup_brak_xlen1)\
             .workplane(origin = (1.5*sup_brak_ylen , 0),
                        offset = enc.outbox_zlen/2 -  shelf3_zoffset - shelf_h3)\
             .rect(loft_bxlen, loft_bylen ).loft(combine =True)
    
res12 = res11.faces(">>X[-12]").workplane(0,0)\
                               .center(0, shelf_h/2 + (enc.outbox_zlen/2 - shelf3_zoffset
                                                       - shelf_h3)).hole(6)\
                               .faces("<X").workplane().circle(3.05).cutBlind(-shelf_b2 - wall_t)

# #------------------------------------------------------------------------------------------   
# ## Features on GP shelf 1.5 cm pitched breadboard

res13 = res12.faces(">>Z[16]").first()\
             .workplane(origin = (enc.outbox_xlen/2 - shelf_b2/2 - wall_t/2,0))\
             .rarray(15, 15, 5,14, center=True).hole(3.05)

# #------------------------------------------------------------------------------------------   
# ## Multi hole arrays for 45 degree beam mounts


res14 = res13.faces(">X").workplane(origin =  (0,outbox_ylen/4 + outbox_ylen/8,
                                               outbox_zlen/4+ outbox_zlen/8 - shelf_h))\
             .rarray(15, 15, 3,3, center=True).hole(6.1)\
             .faces(">X").workplane(origin = (0,-outbox_ylen/4 - outbox_ylen/8,
                                              outbox_zlen/4+ outbox_zlen/8 - shelf_h))\
             .rarray(15, 15, 3,3, center=True).hole(6.1)\
             .faces(">X").workplane(origin = (0,outbox_ylen/4 + outbox_ylen/8,
                                              -outbox_zlen/4 - outbox_zlen/8 + shelf_loc))\
             .rarray(15, 15, 3,3, center=True).hole(6.1)\
             .faces(">X").workplane(origin = (0,-outbox_ylen/4 - outbox_ylen/8,
                                              -outbox_zlen/4 - outbox_zlen/8 + shelf_loc))\
             .rarray(15, 15, 3,3, center=True).hole(6.1)

res14 = res14.faces(">X").workplane(origin = (0,0,shelf_loc - shelf_h/2 + 30))\
                         .rarray(15, 15, 1,7, center=False).hole(6.1)
                    

In [8]:
# #------------------------------------------------------------------------------------------   
# ## IEC C14 connector port opening 

C14_hole_diam = 3.2
C14_hole_loc = 40
C14_rect_xlen, C14_rect_ylen  = (36,54)



res15 = res14.faces("<Y").workplane(origin = (-outbox_ylen/4 - outbox_ylen/8 + wall_t/2,
                                              0, -outbox_zlen/4 - outbox_zlen/8 + wall_t)
                                   )\
                         .rect(C14_rect_xlen, C14_rect_ylen ).cutBlind(-wall_t) 

res15 = res15.faces("<Y").workplane().rarray(C14_hole_loc,
                                     C14_rect_ylen/3,
                                     2,1, center=True)\
                 .circle(C14_hole_diam/2).cutBlind(-wall_t)


# #------------------------------------------------------------------------------------------   
# ## Mounting holes for cartridge holder
stand_height = 50
res16 = res15.faces(s(">Y")).workplane(origin = (0,0,0)).rarray(1.5*C14_hole_loc,
                                     C14_rect_ylen/3,
                                     2,1, center=True)\
                            .circle(C14_hole_diam/2).cutBlind(-wall_t)\
                            .faces(s(">Y")).workplane(origin = (0,0,-stand_height))\
                            .rarray(1.5*C14_hole_loc,
                                     C14_rect_ylen/3,
                                     2,1, center=True)\
                            .circle(C14_hole_diam/2).cutBlind(-wall_t)\
                    

# #------------------------------------------------------------------------------------------   
# ## Supporting beams for 4 way split

res17 = res16.faces(">Y").workplane(origin = (0,0,shelf_loc + 3))
res17 = res17.rarray(outbox_xlen- 2.5 * wall_t, C14_rect_ylen/3,
                                     2,1, center=True)\
                 .circle(3).cutBlind(-outbox_ylen)

# #------------------------------------------------------------------------------------------   
# ## Close box to create lids
lid_h = wall_t/2
lip_h = 2
res18 = res17.faces(">Z").workplane(origin = (0,0,0), offset = lid_h/2)\
             .box(outbox_xlen,outbox_ylen,lid_h)
res18 = res18.faces("<Z").workplane(origin = (0,0,0), offset = lid_h/2)\
             .box(outbox_xlen,outbox_ylen,lid_h)

# #------------------------------------------------------------------------------------------   

# ## Holes for column rods, fillets for corners

topBot_rad = 7

res19 = res18.edges(s(">Z") + s("<Z") ).fillet(topBot_rad)

side_rad = 1

res19 = res19.edges(s(">XY") + s("< XY")).fillet(side_rad)
res19 = res19.rotateAboutCenter((0,0,1),90).edges(s(">XY") + s("< XY")).fillet(side_rad)
res19 = res19.rotateAboutCenter((0,0,1),-90)
res19 = res19.faces(">Z").workplane(origin = (0,0,0))\
             .rect(enc.hole_x,enc.hole_y, forConstruction = True)\
             .vertices().hole(6.1)


#------------------------------------------------------------------------------------------   
## Breadboard on base

res20 = res19.faces("<Z").workplane(origin = (0,0,0))\
                         .rect(enc.hole_x,enc.hole_y, forConstruction = True)\
                         .vertices().hole(6.1)
res20 = res19.faces("<Z").workplane(origin = (0,0,0))\
                         .rarray(25,25,8,8).cboreHole(6.1, 8, wall_t/4, depth = wall_t)

#------------------------------------------------------------------------------------------   
## Split box into top and bot

(top,bot) = res20.faces(">Z").workplane(origin = (0,0,0), offset = - lid_h)\
                             .split(keepTop = True, keepBottom = True).all()

#------------------------------------------------------------------------------------------   
## Create lip on lid and translate for showcase

lipHeight = 2
lowerTop = top.translate((0,0,-lipHeight))
boss = lowerTop.cut(bot)
boss = boss.translate((0,outbox_ylen + 20,-outbox_zlen  -lid_h/2 - lipHeight))\
           .rotateAboutCenter((0,1,0),180)

both = boss.union(bot)

#------------------------------------------------------------------------------------------   
## Split into L and R parts

(L,R) = bot.faces(">X").workplane(offset = -(shelf_b + wall_t + 2))\
                       .split(keepTop = True, keepBottom = True).all()

#------------------------------------------------------------------------------------------   

## Split into L and R , up and down, top and bottom parts

(Lup, Ldown) = L.faces(">Y").workplane(origin = (0,0,0), offset =   -54.5)\
                            .split(keepTop = True, keepBottom = True).all()
(Rup, Rdown) =  R.faces(">Y").workplane(origin = (0,0,0), offset =  -outbox_ylen + 54.5)\
                             .split(keepTop = True, keepBottom = True).all()

(LupTop,LupBot) = Lup.faces(">Z").workplane(origin = (0,0,0),
                                            offset = - outbox_zlen/2 + wall_t/4)\
                                 .split(keepTop = True, keepBottom = True).all()
(LdownTop, LdownBot) = Ldown.faces(">Z").workplane(origin = (0,0,0),
                                                   offset = - outbox_zlen/2 + wall_t/4)\
                                        .split(keepTop = True, keepBottom = True).all()
(RdownTop, RdownBot) = Rdown.faces(">Z").workplane(origin = (0,0,0),
                                                   offset = - outbox_zlen/2+ wall_t/4)\
                                        .split(keepTop = True, keepBottom = True).all()
(RupTop, RupBot) = Rup.faces(">Z").workplane(origin = (0,0,0),
                                             offset = - outbox_zlen/2+ wall_t/4)\
                                  .split(keepTop = True, keepBottom = True).all()

In [9]:
#------------------------------------------------------------------------------------------   
## Create assembly
hlg1_b = hlg1.val().BoundingBox().xlen
part1, part2, part3, part4, part5,\
part6, part7, part8 = (LupTop, LupBot, LdownTop, LdownBot,
                       RdownTop, RdownBot, RupTop, RupBot)

part9 = boss
part10 = hlg1.rotateAboutCenter((0,0,1),-90)\
             .translate((-hlg1_height/2+hlg1_laser_e_d,
                        enc.inbox_ylen/2 - hlg1_b/2,
                        laser_z_clr + hlg1_len/2 - cartridge_h))
part11 = cartridge.rotateAboutCenter((0,0,1),-90)\
                  .translate((0, enc.inbox_ylen/2 - hlg1_b + cartridge_l/2,0))


assy0 = (
        cq.Assembly(part1,loc = cq.Location(cq.Vector(0,0,0)))
        .add(part2,loc = cq.Location(cq.Vector(0, 0, 0)))         
        .add(part3,loc = cq.Location(cq.Vector(0, 0, 0)))
        .add(part4,loc = cq.Location(cq.Vector(0, 0, 0)))
        .add(part4,loc = cq.Location(cq.Vector(0, 0, 0)))    
        .add(part5,loc = cq.Location(cq.Vector(0, 0, 0)))
        .add(part6,loc = cq.Location(cq.Vector(0, 0, 0)))
        .add(part7,loc = cq.Location(cq.Vector(0, 0, 0)))    
        .add(part8,loc = cq.Location(cq.Vector(0, 0, 0)))        
        .add(part9,loc = cq.Location(cq.Vector(0, 0, 0)))        
        .add(part10,loc = cq.Location(cq.Vector(0, 0, 0)),
             color=cq.Color(0, 70/255, 100/255, 0.1))     
        .add(part11,loc = cq.Location(cq.Vector(0, 0, 0)),
             color=cq.Color(70/255, 0.1 , 100/255, 0.01))         
        )

assy0

In [10]:
mySTEP_path = "C:\\Users\\Srinivas.G\\OneDrive - Ram Group Global Pte Ltd\\Work_shared\\Technical drawings\\mySTEP\\Spectrometer\\Enclosure\\STEP\\"
cq.importers.importStep(f"{mySTEP_path}Assembly.step")

In [11]:
mySTEP_path = "C:\\Users\\Srinivas.G\\OneDrive - Ram Group Global Pte Ltd\\Work_shared\\Technical drawings\\mySTEP\\\Spectrometer\\Enclosure\\STEP\\"
assy0.save(f"{mySTEP_path}Assembly.step")

names = [f"part{i}" for i in range(12)]
parts = [part for part in assy0.children]

for i in range(len(parts)):
    parts[i].save(f"{mySTEP_path}{names[i]}.step")


In [14]:
parts[6]

In [13]:

    
#     parts[i].exportStl(f"{mySTEP_path}{names[i]}.stl", tolerance=0.001, angularTolerance=0.1)